In [532]:
# using Statistics
import Base
using StaticArrays;

In [503]:
struct Individual
    position
    cost
end

In [504]:
struct Output
    pop
    bestsol
    bestcost
end

In [505]:
struct Problem
    costfunc
    nvar
    varmin
    varmax
end

In [506]:
struct Parameter
    maxit
    npop
    beta
    pc
    gamma
    mu
    sugma
end

In [613]:
function exp_array(x)
    out = zeros(length(x))
    for i in 1:length(x)
        out[i] = exp(x[i])
    end
    return out
end

exp_array (generic function with 1 method)

In [702]:
function roulette_wheel_selection(p)
    c = cumsum(p, dims = 1)
    r = Base.sum(p)*rand()
    ind = 0
    for i in 1:length(c)
        if r <= c[i]
            ind = i
            break
        end
    end
    return ind
end

roulette_wheel_selection (generic function with 1 method)

In [1057]:
function  crossover(p1, p2, gamma=0.1)
    c1 = deepcopy(p1)
    c2 = deepcopy(p1)
    alpha = rand(length(c1.position))*(1+2*gamma).-gamma
    c1 = Individual(alpha.*p1.position + (1 .-alpha).*p2.position, c1.cost)
    c2 = Individual(alpha.*p2.position + (1 .-alpha).*p1.position, c2.cost)
    return c1, c2
end

crossover (generic function with 2 methods)

In [746]:
function mutate(x, mu, sugma)
    y = deepcopy(x)
    ind  = []
    for i in 1:length(x.position)
        if rand()<= mu
            append!(ind,i)
        end
    end
    y.position[ind] += sugma*rand(length(ind))
    return y
end

mutate (generic function with 1 method)

In [801]:
function max_v(array, max)
    for i in 1:length(array)
        if array[i] < max[i]
            array[i] = max[i]
        end
    end
    return array
end

max_v (generic function with 1 method)

In [802]:
function min_v(array, min)
    for i in 1:length(array)
        if array[i] > min[i]
            array[i] = min[i]
        end
    end
    return array
end

min_v (generic function with 1 method)

In [803]:
function apply_bound(x, varmin, varmax)
    x = Individual(max_v(x.position, varmin), x.cost)
    x = Individual(min_v(x.position, varmax), x.cost)
    return x
end

apply_bound (generic function with 1 method)

In [1071]:
function sortbycost(pop)
    for i in 1:length(pop)
        for j in 1:length(pop)
            if pop[i].cost < pop[j].cost
                tmp = pop[j]
                pop[j] = pop[i]
                pop[i] = tmp
            end
        end
    end
    return pop
end

sortbycost (generic function with 1 method)

In [1113]:
function run(problem, params)
    # Problem Information
    costfunc = problem.costfunc
    nvar = problem.nvar
    varmin = problem.varmin
    varmax = problem.varmax

    # Parameters
    maxit = params.maxit
    npop = params.npop
    beta = params.beta
    pc = params.pc
    nc = Int(floor(pc*npop/2)*2)
    gamma = params.gamma
    mu = params.mu
    sugma = params.sugma
    
    # Empty Individual Template
    empty_individual = Individual(Cvoid,Cvoid)
    bestsol = deepcopy(empty_individual)
    # bestsol.cost = Inf
    bestsol = Individual(bestsol.position, Inf)
    
    # Initialize Population
    pop=[]
    for i in 1:npop
       push!(pop,empty_individual)
    end
    for i in 1:npop
        pos = rand(nvar).*(varmax-varmin).+varmin
        costc = costfunc(pos)
        pop[i] = Individual(pos, costc)
        if pop[i].cost < bestsol.cost
            bestsol = deepcopy(pop[i])
        end
    end
    
    # Best Cost of Iterations 
    bestcost = zeros(maxit)
    
    # Main Loop
    for it in 1:maxit
        costs = []
        for i in 1:length(pop)
            append!(costs, pop[i].cost)
        end
        avg_cost = mean(costs)
        if avg_cost != 0
            costs = costs ./avg_cost
        end
        probs = exp_array(-beta.*costs)
        
        popc = []
        for _ in 1:floor(nc/2)
            # Perform Roulette Wheel Selection
            p1 = pop[roulette_wheel_selection(probs)]
            p2 = pop[roulette_wheel_selection(probs)]
            
            # Perform Crossover
            c1, c2 = crossover(p1, p2, gamma)
            
            # Perform Mutation
            c1 = mutate(c1, mu, sugma)
            c2 = mutate(c2, mu, sugma)
            
            # Apply Bounds
            c1 = apply_bound(c1, varmin, varmax)
            c2 = apply_bound(c2, varmin, varmax)
            println("c1 = ",c1.position)
            println("c2 = ",c2.position)
            
            # Evaluate First Offspring
            c1 = Individual(c1.position, costfunc(c1.position))
            if c1.cost < bestsol.cost
                bestsol = deepcopy(c1)
            end

            # Evaluate Second Offspring
            c2 = Individual(c2.position, costfunc(c2.position))
            if c2.cost < bestsol.cost
                bestsol = deepcopy(c2)
            end
            
            # Add Offsprings to popc
            push!(popc,c1)
            push!(popc,c2)
        end
                
        # Merge, Sort and Select 87
        for i in 1:length(popc)
            push!(pop,popc[i])
        end
        pop = sortbycost(pop)
        pop = deepcopy(pop[1:npop])

        # Store Best Cost
        bestcost[it] = bestsol.cost

        # Show Iteration Information
        println("Iteration "*string(it)*": Best Cost = "*string(bestcost[it]))
    end
    # Output
    # out = Output(pop,bestsol,bestcost)
    return bestsol
end

run (generic function with 1 method)

In [1114]:
function sphere(x)
    return Base.sum(x.^2)
end

sphere (generic function with 1 method)

In [1115]:
problem = Problem(sphere, 5, [-10, -10, -1, -5,  4],[ 10,  10,  1,  5, 10])
params  = Parameter(100, 50, 1, 1, 0.1, 0.01, 0.1)

Parameter(100, 50, 1, 1, 0.1, 0.01, 0.1)

In [1116]:
out = run(problem, params)

c1 = [-0.9222281403923671, 2.5527404601710115, 0.5344964402368375, -4.599372119383584, 8.27903221019974]
c2 = [1.1659642530059675, 3.2954490034070583, 0.6265330942866241, -4.193548101297697, 8.808583991747193]
c1 = [1.4108678429688353, -7.6317516968962975, -0.19543662374015452, -1.4760703079499502, 10.0]
c2 = [-5.15724839134022, -5.274376498422375, 0.4217244410413356, -1.4852250007899939, 5.984815175589754]
c1 = [3.8638781426245465, -0.35623832827144275, 0.5860659189692298, 1.549645475384556, 6.122599549541526]
c2 = [5.459839126091762, 3.563627013671446, 0.16425774361228912, -1.7817613273915494, 6.596924603115468]
c1 = [-3.9321741048200387, -2.8199096232964815, 0.16046808414523972, 2.5649331629617445, 4.5707469374636895]
c2 = [6.810713159924946, -0.5638574841306885, -0.35827665331326897, 3.72370219936971, 9.373467461685243]
c1 = [-6.607594995578344, -2.921634043093156, 0.18334685641205445, 0.7815652978728033, 6.422676538414936]
c2 = [-4.97117413873922, -5.3306891853702005, -0.702150175

Excessive output truncated after 524289 bytes.

c1 = [-0.000982738032190609, -0.00018278179566283209, 0.0009643059861070916, 0.26084124829384625, 4.6623304825558645]
c2 = [-0.0015682783797185125, -0.000147483267059038, 0.0009547633992331632, 0.2608441514477235, 4.662330494635043]
c1 = [-0.0012300811208580213, -5.9702734184603756e-5, 0.0014248564059458676, 0.2608424276124151, 4.662330479760886]
c2 = [-0.0011430965130768919, -0.0001642640485512265, 0.0014247208419761077, 0.26084411568980603, 4.6623303310342905]
c1 = [-0.0015242133762940913, -0.00012686375161110195, 0.0012044394086867464, 0.26084146629855215, 4.662330453030281]
c2 = [-0.0015314689754047186, -0.0001974733450466143, 0.0010356537882778697, 0.2608402707068546, 4.662330511007859]
c1 = [-0.0011583618960805653, -0.00016235957209038363, 0.001095377991213105, 0.26084302496614437, 4.662330348430706]
c2 = [-0.0015176324070727333, -0.00011841252675074703, 0.000844651347438826, 0.2608420221647826, 4.662330391069527]
c1 = [-0.0012190571059901326, -0.00012219940618002183, 0.001398995

Individual([-0.0010329451737208762, -0.00013375063555169566, 0.0009762317430943088, 0.2608361361771623, 4.662330365676463], 21.805361966538033)

In [1103]:
out.cost

17.058692677639076